# Advanced OpenAI Agents with Tools

This notebook demonstrates more advanced usage of OpenAI Agents with custom tools and multi-agent workflows.

## Setup

First, let's install the required packages and set up our environment:

In [ ]:
# Install required packages
# %pip install openai python-dotenv requests pydantic

In [ ]:
import os
import json
import requests
from typing import Dict, List, Optional, Any, Union
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

print("Environment initialized!")

## 1. Creating Structured Tools

Let's define some custom tools with proper schema definitions using Pydantic:

In [ ]:
# Define tool schemas
class SearchQuery(BaseModel):
    """Search query parameters"""
    query: str = Field(..., description="The search query string")
    max_results: int = Field(5, description="Maximum number of results to return")

class WeatherRequest(BaseModel):
    """Weather request parameters"""
    location: str = Field(..., description="City name or location")
    units: str = Field("metric", description="Units: 'metric' for Celsius, 'imperial' for Fahrenheit")
    
class TextAnalysisRequest(BaseModel):
    """Text analysis parameters"""
    text: str = Field(..., description="The text to analyze")
    analysis_type: str = Field(..., description="Type of analysis: 'sentiment', 'entities', or 'summary'")

In [ ]:
# Implement tool functions
def search_web(params: Dict[str, Any]) -> str:
    """Mock web search function"""
    query = params.get("query", "")
    max_results = params.get("max_results", 5)
    
    # In a real implementation, this would call a search API
    results = [
        {"title": f"Result {i} for {query}", "snippet": f"This is a snippet for result {i}"}
        for i in range(1, min(max_results + 1, 6))
    ]
    
    return json.dumps(results, indent=2)

def get_weather(params: Dict[str, Any]) -> str:
    """Mock weather information function"""
    location = params.get("location", "")
    units = params.get("units", "metric")
    
    temp = 22 if units == "metric" else 72
    unit_symbol = "°C" if units == "metric" else "°F"
    
    # In a real implementation, this would call a weather API
    weather_data = {
        "location": location,
        "temperature": f"{temp}{unit_symbol}",
        "conditions": "Partly cloudy",
        "humidity": "65%",
        "wind": "10 km/h"
    }
    
    return json.dumps(weather_data, indent=2)

def analyze_text(params: Dict[str, Any]) -> str:
    """Mock text analysis function"""
    text = params.get("text", "")
    analysis_type = params.get("analysis_type", "sentiment")
    
    if analysis_type == "sentiment":
        # In a real implementation, this would use NLP
        return json.dumps({"sentiment": "positive", "confidence": 0.85})
    elif analysis_type == "entities":
        # Extract random entities as example
        words = text.split()
        entities = [words[i] for i in range(0, len(words), 3) if i < len(words)]
        return json.dumps({"entities": entities})
    elif analysis_type == "summary":
        # Create a mock summary
        return json.dumps({"summary": f"Summary of: {text[:50]}..."})
    else:
        return json.dumps({"error": "Invalid analysis type"})

In [ ]:
# Define the tools in OpenAI's format
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for information on a given topic",
            "parameters": SearchQuery.schema()
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather information for a location",
            "parameters": WeatherRequest.schema()
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_text",
            "description": "Analyze text for sentiment, entities, or generate a summary",
            "parameters": TextAnalysisRequest.schema()
        }
    }
]

## 2. Advanced Agent with Multiple Tools

Now let's create an advanced agent that can use multiple tools efficiently:

In [ ]:
def run_advanced_agent(user_query):
    """Run an advanced agent with access to multiple tools"""
    # Initial system message defines agent capabilities and behavior
    messages = [
        {
            "role": "system",
            "content": """
            You are an advanced AI assistant with access to multiple tools.
            You can search the web, check weather, and analyze text.
            Use these tools to provide comprehensive, accurate answers.
            When using multiple tools, organize your responses clearly.
            Always cite your sources when providing factual information.
            """
        },
        {"role": "user", "content": user_query}
    ]
    
    print(f"\nProcessing query: {user_query}")
    print("\nThinking...")
    
    # We'll allow multiple tool call cycles for complex tasks
    max_turns = 3
    current_turn = 0
    
    while current_turn < max_turns:
        current_turn += 1
        
        # Get assistant's response with potential tool calls
        response = client.chat.completions.create(
            model="gpt-4o",  # Using GPT-4o for advanced capabilities
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        assistant_response = response.choices[0].message
        messages.append(assistant_response)
        
        # If no tool calls or final response, we're done
        if not assistant_response.tool_calls:
            print(f"\nFinal Response (after {current_turn} turns):")
            print(assistant_response.content)
            break
            
        # Process tool calls
        print(f"\nTurn {current_turn}: Processing {len(assistant_response.tool_calls)} tool calls...")
        
        for tool_call in assistant_response.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"  - Executing {function_name} with args: {function_args}")
            
            # Call the appropriate function
            if function_name == "search_web":
                function_response = search_web(function_args)
            elif function_name == "get_weather":
                function_response = get_weather(function_args)
            elif function_name == "analyze_text":
                function_response = analyze_text(function_args)
            else:
                function_response = json.dumps({"error": f"Unknown function {function_name}"})
            
            # Add tool response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
    
    # Get final response if we exited due to max turns
    if current_turn == max_turns and assistant_response.tool_calls:
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        print(f"\nFinal Response (after maximum {max_turns} turns):")
        print(final_response.choices[0].message.content)
    
    return messages

In [ ]:
# Test with a complex query that may require multiple tools
complex_query = "What's the weather in Tokyo right now, and can you find information about popular tourist attractions there? Also, analyze the sentiment of this review: 'The hotel was amazing with great views, but the staff was somewhat unfriendly.'"

# Run the agent
conversation = run_advanced_agent(complex_query)

## 3. Specialized Agent for Data Analysis

Let's create a specialized agent for data analysis tasks:

In [ ]:
class DataAnalysisRequest(BaseModel):
    """Parameters for data analysis"""
    data: str = Field(..., description="JSON string containing the data to analyze")
    analysis_type: str = Field(..., description="Type of analysis: 'statistics', 'trends', or 'visualization'")
    
def analyze_data(params: Dict[str, Any]) -> str:
    """Mock data analysis function"""
    try:
        data_str = params.get("data", "[]")
        analysis_type = params.get("analysis_type", "statistics")
        
        # Parse the data (assuming it's JSON)
        data = json.loads(data_str)
        
        if analysis_type == "statistics":
            # Mock statistics analysis
            result = {
                "count": len(data) if isinstance(data, list) else 1,
                "type": str(type(data)),
                "sample": str(data)[:100] + "..."
            }
        elif analysis_type == "trends":
            # Mock trend analysis
            result = {
                "trend": "upward",
                "confidence": 0.75,
                "note": "This is a mock trend analysis"
            }
        elif analysis_type == "visualization":
            # Mock visualization suggestion
            result = {
                "suggested_visualization": "bar chart",
                "alternative": "line graph",
                "note": "This is a mock visualization suggestion"
            }
        else:
            result = {"error": "Invalid analysis type"}
            
        return json.dumps(result, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)}, indent=2)

# Add this tool to our tools list
data_analysis_tool = {
    "type": "function",
    "function": {
        "name": "analyze_data",
        "description": "Analyze data for statistics, trends, or visualization suggestions",
        "parameters": DataAnalysisRequest.schema()
    }
}

# Create a specialized set of tools for the data analysis agent
data_analysis_tools = [data_analysis_tool]

In [ ]:
def run_data_analysis_agent(data_query):
    """Run a specialized data analysis agent"""
    messages = [
        {
            "role": "system",
            "content": """
            You are a specialized data analysis assistant.
            You help users understand data, generate statistics, identify trends, 
            and suggest appropriate visualizations.
            Always request the data in a proper format, preferably JSON.
            Provide clear explanations of your findings.
            """
        },
        {"role": "user", "content": data_query}
    ]
    
    print(f"\nProcessing data query: {data_query}")
    
    # Get initial response
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=data_analysis_tools,
        tool_choice="auto"
    )
    
    assistant_response = response.choices[0].message
    messages.append(assistant_response)
    
    print("\nInitial response:")
    if hasattr(assistant_response, 'content') and assistant_response.content:
        print(assistant_response.content)
    
    # Process any tool calls
    if assistant_response.tool_calls:
        for tool_call in assistant_response.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"\nCalling {function_name}...")
            
            # Execute the function
            if function_name == "analyze_data":
                function_response = analyze_data(function_args)
            else:
                function_response = json.dumps({"error": f"Unknown function {function_name}"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
        
        # Get final response
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        print("\nAnalysis results:")
        print(final_response.choices[0].message.content)
        return final_response.choices[0].message.content
    else:
        return assistant_response.content

In [ ]:
# Test with some sample data
sample_data_query = """Can you analyze this sales data and suggest a good visualization?
```json
[
  {"month": "January", "sales": 10000},
  {"month": "February", "sales": 12000},
  {"month": "March", "sales": 18000},
  {"month": "April", "sales": 15000},
  {"month": "May", "sales": 20000}
]
```
"""

data_analysis_result = run_data_analysis_agent(sample_data_query)

## 4. Multi-Agent Collaboration

Let's demonstrate how to implement a simple multi-agent system where agents collaborate:

In [ ]:
def create_agent_with_persona(persona, available_tools):
    """Create an agent with a specific persona and toolset"""
    
    def agent_function(query):
        messages = [
            {"role": "system", "content": persona},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=available_tools,
            tool_choice="auto"
        )
        
        assistant_response = response.choices[0].message
        messages.append(assistant_response)
        
        # Process any tool calls
        if assistant_response.tool_calls:
            for tool_call in assistant_response.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                # Map function names to functions
                function_map = {
                    "search_web": search_web,
                    "get_weather": get_weather,
                    "analyze_text": analyze_text,
                    "analyze_data": analyze_data
                }
                
                if function_name in function_map:
                    function_response = function_map[function_name](function_args)
                else:
                    function_response = json.dumps({"error": f"Unknown function {function_name}"})
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                })
            
            # Get final response
            final_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            return final_response.choices[0].message.content
        else:
            return assistant_response.content
    
    return agent_function

In [ ]:
# Create specialized agents
research_agent = create_agent_with_persona(
    "You are a research specialist. Your job is to find information on topics. Focus only on facts and be concise.",
    [tools[0]]  # Only search_web tool
)

analysis_agent = create_agent_with_persona(
    "You are a text analysis specialist. You analyze text for sentiment, extract entities, and create summaries.",
    [tools[2]]  # Only analyze_text tool
)

weather_agent = create_agent_with_persona(
    "You are a weather specialist. You provide detailed weather information for locations.",
    [tools[1]]  # Only get_weather tool
)

In [ ]:
def multi_agent_workflow(query):
    """Run a workflow with multiple specialized agents"""
    print(f"\nProcessing: {query}")
    
    # First, use the main agent to decide which specialized agents to use
    messages = [
        {
            "role": "system",
            "content": """
            You are a coordinator agent. Your job is to analyze the user query and decide which
            specialized agents to call. You have access to these agents:
            1. Research Agent - For finding information on topics
            2. Analysis Agent - For analyzing text sentiment, entities, summaries
            3. Weather Agent - For weather information
            
            Return your decision as a JSON object with the agents to call and sub-queries for each.
            """
        },
        {"role": "user", "content": query}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"}
    )
    
    plan = json.loads(response.choices[0].message.content)
    print("\nCoordinator's plan:")
    print(json.dumps(plan, indent=2))
    
    # Execute the plan by calling each agent
    results = {}
    
    if plan.get("research_agent"):
        print("\nCalling Research Agent...")
        results["research"] = research_agent(plan["research_agent"])
        
    if plan.get("analysis_agent"):
        print("\nCalling Analysis Agent...")
        results["analysis"] = analysis_agent(plan["analysis_agent"])
        
    if plan.get("weather_agent"):
        print("\nCalling Weather Agent...")
        results["weather"] = weather_agent(plan["weather_agent"])
    
    # Synthesize the results
    synthesis_prompt = f"""
    Original query: {query}
    
    Results from specialized agents:
    {json.dumps(results, indent=2)}
    
    Please synthesize these results into a comprehensive, well-organized response.
    """
    
    messages = [
        {
            "role": "system",
            "content": "You are a synthesis agent. Your job is to combine results from multiple specialized agents into a coherent response."
        },
        {"role": "user", "content": synthesis_prompt}
    ]
    
    final_response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    
    print("\nFinal Synthesized Response:")
    print(final_response.choices[0].message.content)
    
    return {
        "plan": plan,
        "individual_results": results,
        "synthesized_response": final_response.choices[0].message.content
    }

In [ ]:
# Test the multi-agent workflow
complex_query = "I'm planning a trip to Paris next week. What's the weather like there, and what are some must-see attractions? Also, can you analyze this hotel review: 'The rooms were spacious and the location was perfect, but the breakfast was disappointing.'"

workflow_results = multi_agent_workflow(complex_query)

## Conclusion

This notebook has demonstrated advanced techniques for working with OpenAI Agents:

1. **Structured Tools with Pydantic** - Creating well-defined tool schemas
2. **Advanced Multi-Tool Agents** - Agents that can use multiple tools in sequence
3. **Specialized Agents** - Creating agents for specific tasks like data analysis
4. **Multi-Agent Workflows** - Coordinating multiple agents to solve complex problems

These patterns can be extended to build sophisticated AI applications that leverage the strengths of multiple specialized components.